# Project 3 - Part 3 (Core)

The project assignment is at the beginning of this week because you already have all of the background to complete project part 3 based on the first two weeks of the course!
Business Problem

    For this project, you have been hired to produce a MySQL database on Movies from a subset of IMDB's publicly available dataset. Ultimately, you will use this database to analyze what makes a movie successful, and will provide recommendations to the stakeholder on how to make a successful movie.

Over the course of this project, you will:

    Part 1: Download several files from IMDB’s movie data set and filter out the subset of moves requested by the stakeholder.
    Part 2: Use an API to extract box office revenue and profit data to add to your IMDB data and perform exploratory data analysis.
    Part 3: Construct and export a MySQL database using your data.
    Part 4: Apply hypothesis testing to explore what makes a movie successful.
    Part 5 (Optional): Produce a Linear Regression model to predict movie performance.

## Part 3

    For part 3 of the project you will be practicing applying an E.T.L process on your previously saved movie data. Specifically, you will create a new MySQL database after preparing the data for a relational database. You will export your database to a .sql file in your repository using MySQL Workbench.

Specifications - Database

    Your stakeholder wants you to take the data you have been cleaning and collecting in Parts 1 & 2 of the project, and wants you to create a MySQL database for them.

    Specifically, they want the data from the following files included in your database:
        Title Basics:
            Movie ID (tconst)
            Primary Title
            Start Year
            Runtime (in Minutes)
            Genres
        Title Ratings
            Movie ID (tconst)
            Average Movie Rating
            Number of Votes
        The TMDB API Results (multiple files)
            Movie ID
            Revenue
            Budget
            Certification (MPAA Rating)

    You should normalize the tables as best you can before adding them to your new database.
        Note: an important exception to their request is that they would like you to keep all of the data from the TMDB API in 1 table together (even though it will not be perfectly normalized).
        You only need to keep the imdb_id, revenue, budget, and certification columns


## Required Transformation Steps for Title Basics:

    Normalize Genre:
        Convert the single string of genres from title basics into 2 new tables.

            title_genres: with the columns:
                tconst
                genre_id

            genres:
                genre_id
                genre_name

    Discard unnecessary information:
        For the title basics table, drop the following columns:
            "original_title" (we will use the primary title column instead)
            "isAdult" ("Adult" will show up in the genres so this is redundant information).
            "titleType" (every row will be a movie).
            "genres" and other variants of genre (genre is now represented in the 2 new tables described above.
        Do not include the title_akas table in your SQL database.
            You have already filtered out the desired movies using this table and the remaining data is mostly nulls and not of-interest to the stakeholder.

MySQL Database Requirements

    Use sqlalchemy with pandas to execute your SQL queries inside your notebook.

    Create a new database on your MySQL server and call it "movies".

    Make sure to have the following tables in your "movies" database:
        title_basics
        title_ratings
        title_genres
        genres
        tmdb_data

    Make sure to set a Primary Key for each table that isn't a joiner table (e.g. title_genres is a joiner table).

    After creating each table, show the first 5 rows of that table using a SQL query.

    Make sure to run the "SHOW TABLES" SQL query at the end of your notebook to show that all required tables have been created.

## Title Basics:

    Movie ID (tconst)
    Primary Title
    Start Year
    Runtime (in Minutes)
    Genres

In [1]:
import os, time,json
import tmdbsimple as tmdb 
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)
import pandas as pd
from tqdm.notebook import tqdm_notebook
import tmdbsimple as tmdb
from yelpapi import YelpAPI

In [2]:
with open('C:/Users/tulan/.secret/TMDB_api.json', 'r') as f:
    login = json.load(f)

tmdb.API_KEY =  login['api-key']

In [4]:
basics = pd.read_csv('Data/final_basics.csv.gz')
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama
...,...,...,...,...,...,...,...,...,...
84554,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,NaN,74,Drama
84555,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019,NaN,97,"Comedy,Drama,Fantasy"
84556,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama
84557,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller"


In [5]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84559 entries, 0 to 84558
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          84559 non-null  object 
 1   titleType       84559 non-null  object 
 2   primaryTitle    84559 non-null  object 
 3   originalTitle   84559 non-null  object 
 4   isAdult         84559 non-null  int64  
 5   startYear       84559 non-null  int64  
 6   endYear         0 non-null      float64
 7   runtimeMinutes  84559 non-null  int64  
 8   genres          84559 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 5.8+ MB


### Drop unnecessary columns

In [10]:
 = basics.drop(['originalTitle','isAdult','titleType'], axis=1)
basics

,tconst,primaryTitle,startYear,endYear,runtimeMinutes,genres
0,tt0035423,Kate & Leopold,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,The Tango of the Widower and Its Distorting Mi...,2020,NaN,70,Drama
2,tt0069049,The Other Side of the Wind,2018,NaN,122,Drama
3,tt0088751,The Naked Monster,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,Crime and Punishment,2002,NaN,126,Drama
...,...,...,...,...,...,...
84554,tt9914942,Life Without Sara Amat,2019,NaN,74,Drama
84555,tt9915872,The Last White Witch,2019,NaN,97,"Comedy,Drama,Fantasy"
84556,tt9916170,The Rehearsal,2019,NaN,51,Drama
84557,tt9916190,Safeguard,2020,NaN,95,"Action,Adventure,Thriller"


### 1. Getting List of Unique Genres

In [12]:
## create a col with a list of genres
basics['genres_split'] = basics['genres'].str.split(',')
basics



,tconst,primaryTitle,startYear,endYear,runtimeMinutes,genres,genres_split
0,tt0035423,Kate & Leopold,2001,NaN,118,"Comedy,Fantasy,Romance","[Comedy, Fantasy, Romance]"
1,tt0062336,The Tango of the Widower and Its Distorting Mi...,2020,NaN,70,Drama,[Drama]
2,tt0069049,The Other Side of the Wind,2018,NaN,122,Drama,[Drama]
3,tt0088751,The Naked Monster,2005,NaN,100,"Comedy,Horror,Sci-Fi","[Comedy, Horror, Sci-Fi]"
4,tt0096056,Crime and Punishment,2002,NaN,126,Drama,[Drama]
...,...,...,...,...,...,...,...
84554,tt9914942,Life Without Sara Amat,2019,NaN,74,Drama,[Drama]
84555,tt9915872,The Last White Witch,2019,NaN,97,"Comedy,Drama,Fantasy","[Comedy, Drama, Fantasy]"
84556,tt9916170,The Rehearsal,2019,NaN,51,Drama,[Drama]
84557,tt9916190,Safeguard,2020,NaN,95,"Action,Adventure,Thriller","[Action, Adventure, Thriller]"


In [13]:
exploded_genres = basics.explode('genres_split')
exploded_genres



,tconst,primaryTitle,startYear,endYear,runtimeMinutes,genres,genres_split
0,tt0035423,Kate & Leopold,2001,NaN,118,"Comedy,Fantasy,Romance",Comedy
0,tt0035423,Kate & Leopold,2001,NaN,118,"Comedy,Fantasy,Romance",Fantasy
0,tt0035423,Kate & Leopold,2001,NaN,118,"Comedy,Fantasy,Romance",Romance
1,tt0062336,The Tango of the Widower and Its Distorting Mi...,2020,NaN,70,Drama,Drama
2,tt0069049,The Other Side of the Wind,2018,NaN,122,Drama,Drama
...,...,...,...,...,...,...,...
84557,tt9916190,Safeguard,2020,NaN,95,"Action,Adventure,Thriller",Action
84557,tt9916190,Safeguard,2020,NaN,95,"Action,Adventure,Thriller",Adventure
84557,tt9916190,Safeguard,2020,NaN,95,"Action,Adventure,Thriller",Thriller
84558,tt9916362,Coven,2020,NaN,92,"Drama,History",Drama


In [16]:
genres_split = basics['genres'].str.split(",")

unique_genres = genres_split.explode().unique()
unique_genres

array(['Comedy', 'Fantasy', 'Romance', 'Drama', 'Horror', 'Sci-Fi',
       'Mystery', 'Musical', 'Action', 'Adventure', 'Crime', 'Thriller',
       'Music', 'Animation', 'Family', 'History', 'War', 'Biography',
       'Sport', 'Western', 'Adult', 'Short', 'Reality-TV', 'News',
       'Talk-Show', 'Game-Show'], dtype=object)

In [17]:
unique_genres = sorted(exploded_genres['genres_split'].unique())
unique_genres


['Action',
 'Adult',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Drama',
 'Family',
 'Fantasy',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Short',
 'Sport',
 'Talk-Show',
 'Thriller',
 'War',
 'Western']

## Title Ratings

In [18]:
ratings = pd.read_csv('Data/final_ratings.csv.gz')
ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,1930
1,tt0000002,5.8,261
2,tt0000005,6.2,2560
3,tt0000006,5.1,176
4,tt0000007,5.4,798
...,...,...,...
479650,tt9916204,8.2,251
479651,tt9916348,8.5,17
479652,tt9916362,6.4,5073
479653,tt9916428,3.8,14


In [19]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 479655 entries, 0 to 479654
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         479655 non-null  object 
 1   averageRating  479655 non-null  float64
 2   numVotes       479655 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 11.0+ MB


No changes needed.

## The TMDB API Results

In [20]:
results = pd.read_csv('Data/tmdb_results_combined.csv.gz')
results

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0035423,0.0,/hfeiSfWYujh6MKhtGTXyK3DD4nN.jpg,NaN,48000000.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 14, ...",NaN,11232.0,en,Kate & Leopold,...,76019048.0,118.0,"[{'english_name': 'French', 'iso_639_1': 'fr',...",Released,"If they lived in the same century, they'd be p...",Kate & Leopold,0.0,6.3,1170.0,PG-13
2,tt0114447,0.0,NaN,NaN,0.0,"[{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...",NaN,151007.0,en,The Silent Force,...,0.0,90.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,They left him for dead... They should have fin...,The Silent Force,0.0,5.0,3.0,NaN
3,tt0118589,0.0,/9NZAirJahVilTiDNCHLFcdkwkiy.jpg,NaN,22000000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10402, 'n...",NaN,10696.0,en,Glitter,...,5271666.0,104.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,"In music she found her dream, her love, herself.",Glitter,0.0,4.6,122.0,PG-13
4,tt0118652,0.0,/mWxJEFRMvkG4UItYJkRDMgWQ08Y.jpg,NaN,1000000.0,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",NaN,17140.0,en,The Attic Expeditions,...,0.0,100.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,His search for peace of mind... will leave his...,The Attic Expeditions,0.0,5.1,29.0,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2502,tt6174238,0.0,NaN,NaN,0.0,"[{'id': 80, 'name': 'Crime'}]",NaN,223878.0,cn,冷战,...,0.0,0.0,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,NaN,Cold War,0.0,2.0,2.0,NaN
2503,tt7029820,0.0,NaN,NaN,7000.0,[],NaN,604889.0,en,Scream For Christmas,...,0.0,80.0,[],Released,NaN,Scream For Christmas,0.0,0.0,0.0,NaN
2504,tt7197642,0.0,NaN,NaN,0.0,"[{'id': 35, 'name': 'Comedy'}]",NaN,872676.0,en,"Goodbye, Merry-Go-Round",...,0.0,90.0,[],Released,NaN,"Goodbye, Merry-Go-Round",0.0,0.0,0.0,NaN
2505,tt7631368,0.0,/sF0gUHE0YzZNXYugTB2LFxJIppf.jpg,NaN,10000000.0,"[{'id': 27, 'name': 'Horror'}]",NaN,97186.0,fr,"I, Vampire",...,0.0,85.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,"I, Vampire",0.0,6.4,4.0,NR


In [21]:
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2507 entries, 0 to 2506
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_id                2507 non-null   object 
 1   adult                  2505 non-null   float64
 2   backdrop_path          1334 non-null   object 
 3   belongs_to_collection  201 non-null    object 
 4   budget                 2505 non-null   float64
 5   genres                 2505 non-null   object 
 6   homepage               169 non-null    object 
 7   id                     2505 non-null   float64
 8   original_language      2505 non-null   object 
 9   original_title         2505 non-null   object 
 10  overview               2455 non-null   object 
 11  popularity             2505 non-null   float64
 12  poster_path            2242 non-null   object 
 13  production_companies   2505 non-null   object 
 14  production_countries   2505 non-null   object 
 15  rele